## Work
1. 試比較有 BN 在 Batch_size = 2, 16, 32, 128, 256 下的差異
2. 請嘗試將 BN 放在 Activation 之前，並比較訓練結果
3. 請於 BN 放在 Input Layer 後，並比較結果

In [2]:
import os
import keras
# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:
train, test = keras.datasets.cifar10.load_data()

In [4]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [5]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [17]:
from keras.layers import BatchNormalization
from keras import activations
"""
建立神經網路，並加入 BN layer
"""
def build_mlp_before(input_shape, output_units=10, num_neurons=[512, 256, 128,64,32]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
            x = keras.layers.relU(x,max_value=None)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x,max_value=None)
            x = keras.layers.relU(x)
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model
"""
def build_mlp_after(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model
"""

'\ndef build_mlp_after(input_shape, output_units=10, num_neurons=[512, 256, 128]):\n    input_layer = keras.layers.Input(input_shape)\n    \n    for i, n_units in enumerate(num_neurons):\n        if i == 0:\n            x = keras.layers.Dense(units=n_units, \n                                   activation="relu", \n                                   name="hidden_layer"+str(i+1))(input_layer)\n            x = BatchNormalization()(x)\n        else:\n            x = keras.layers.Dense(units=n_units, \n                                   activation="relu", \n                                   name="hidden_layer"+str(i+1))(x)\n            x = BatchNormalization()(x)\n    \n    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)\n    \n    model = keras.models.Model(inputs=[input_layer], outputs=[out])\n    return model\n'

In [7]:
"""Code Here
設定超參數
"""

LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = [2, 16, 32, 128, 256]
MOMENTUM = 0.95

In [18]:
#results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""

model = build_mlp_before(input_shape=x_train.shape[1:])
model.summary()
optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE[0], 
          validation_data=(x_test, y_test), 
          shuffle=True)

# Collect results
train_loss_before = model.history.history["loss"]
valid_loss_before = model.history.history["val_loss"]
train_acc_before = model.history.history["accuracy"]
valid_acc_before = model.history.history["val_accuracy"]

model = build_mlp_after(input_shape=x_train.shape[1:])
model.summary()
optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE[0], 
          validation_data=(x_test, y_test), 
          shuffle=True)

# Collect results
train_loss_after = model.history.history["loss"]
valid_loss_after = model.history.history["val_loss"]
train_acc_after = model.history.history["accuracy"]
valid_acc_after = model.history.history["val_accuracy"]

AttributeError: module 'keras.layers' has no attribute 'relU'

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(range(len(train_loss_before)), train_loss_before, label="train loss")
plt.plot(range(len(valid_loss_before), valid_loss_before, label="valid loss")

plt.plot(range(len(train_loss_after)), train_loss_after, label="train loss")
plt.plot(range(len(valid_loss_after)), valid_loss_after, label="valid loss")
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(range(len(train_acc_before)), train_acc_before, label="train accuracy")
plt.plot(range(len(valid_acc_before)), valid_acc_before, label="valid accuracy")

plt.plot(range(len(train_acc_after)), train_acc_after, label="train accuracy")
plt.plot(range(len(valid_acc_after)), valid_acc_after, label="valid accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()